# Use examples of [edges](https://github.com/romainsacchi/edges)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `edge` to use exchange-specific
characterization factors in the characterization matrix of `bw2calc`, combining the use of exchange names and custom functions.

## Requirements

* **Pyhton 3.10 or higher (up to 3.11) is highly recommended**

# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, it requires that you have an activated `brightway2` project with a `biosphere3` database as well as an [ecoinvent](https://ecoinvent.prg) v.3 cut-off or consequential database registered in that project. Please refer to the brightway [documentation](https://brightway.dev) if you do not know how to create a project and install ecoinvent.

In [1]:
from edges import EdgeLCIA, get_available_methods
import bw2data

In this example, we will consider a more complex use of parameters based on thre previous example (saved under `lcia_example_4.json`) together with user-defined functions.
Here, we define the CF for methane and dinitrogen monoxide based on `H` (time horizon) and the gas concentration.

In [2]:
import numpy as np

# Physical constants
M_atm = 5.15e18  # kg, total mass of Earth's atmosphere
M_air = 28.96    # g/mol, average molar mass of air

# Gas-specific molecular weights (g/mol)
M_gas = {
    'CO2': 44.01,
    'CH4': 16.04,
    'N2O': 44.013
}

# IPCC concentration parameters (Myhre et al. 1998 / IPCC AR6)
RF_COEFF = {
    'CH4': 0.036,  # W·m⁻²·ppb⁻½ for CH4
    'N2O': 0.12    # W·m⁻²·ppb⁻½ for N2O
}

# Reference atmospheric concentrations (IPCC AR6, ~2019)
C_REF = {
    'CH4': 1866,  # ppb
    'N2O': 332    # ppb
}

# Indirect forcing factor for methane (IPCC AR6)
INDIRECT_FACTOR = {
    'CH4': 1.65,
    'N2O': 1.0
}

# Gas-specific atmospheric lifetimes (years, IPCC AR6)
TAU_GAS = {
    'CH4': 11.8,
    'N2O': 109
}

# CO2 impulse response function parameters (IPCC AR5/AR6)
CO2_IRF = {
    'a0': 0.2173,
    'a': [0.2240, 0.2824, 0.2763],
    'tau': [394.4, 36.54, 4.304]
}

# Convert concentration-based radiative efficiency to mass-based (W·m⁻²·kg⁻¹)
def convert_ppb_to_mass_rf(a_ppb, gas):
    return a_ppb * (M_atm / M_gas[gas]) * (M_air / 1e9)

# Calculate concentration-dependent radiative efficiency
def radiative_efficiency_concentration(gas, concentration_ppb):
    alpha = RF_COEFF[gas]
    return (alpha / (2 * np.sqrt(concentration_ppb))) * INDIRECT_FACTOR[gas]

# AGWP for CO2 (mass-based)
def AGWP_CO2(H):
    integral_CO2 = CO2_IRF['a0'] * H + sum(
        a * tau * (1 - np.exp(-H / tau))
        for a, tau in zip(CO2_IRF['a'], CO2_IRF['tau'])
    )
    am_CO2 = convert_ppb_to_mass_rf(1.37e-5, 'CO2')  # fixed IPCC radiative efficiency for CO2
    return am_CO2 * integral_CO2

# AGWP for gas at given concentration
def AGWP_gas(gas, H, concentration_ppb):
    aa_gas = radiative_efficiency_concentration(gas, concentration_ppb)
    am_gas = convert_ppb_to_mass_rf(aa_gas, gas)
    tau_gas = TAU_GAS[gas]
    return am_gas * tau_gas * (1 - np.exp(-H / tau_gas))

# Calculate concentration-dependent GWP
def GWP(gas, H, concentration_ppb):
    AGWP_g = AGWP_gas(gas, H, concentration_ppb)
    AGWP_ref = AGWP_CO2(H)
    return AGWP_g / AGWP_ref


In [3]:
GWP('CH4', 100, 1911)

30.660731597024444

We do not get exactly 28 for H=100, as the IPCC does, because there are some carbon feedback loops and other kind of complex and non-linear interactions that are not considered in this simple model.

Our JSON file looks like this below. This time, we call `GWP()` and pass it a few parameters (e.g., gas type, time horizon, and gas concentration).

In [1]:
[
  {
    "supplier": {
      "name": "Carbon dioxide",
      "operator": "startswith",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process"
    },
    "value": "1.0"
  },
  {
      "supplier": {
        "name": "Methane, fossil",
        "operator": "contains",
        "matrix": "biosphere"
      },
      "consumer": {
        "matrix": "technosphere",
        "type": "process"
      },
      "value": "GWP('CH4',H, C_CH4)"
    },
  {
    "supplier": {
      "name": "Dinitrogen monoxide",
      "operator": "equals",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process"
    },
    "value": "GWP('N2O',H, C_N2O)"
  }
]

[{'supplier': {'name': 'Carbon dioxide',
   'operator': 'startswith',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process'},
  'value': '1.0'},
 {'supplier': {'name': 'Methane, fossil',
   'operator': 'contains',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process'},
  'value': '28 * (1 + 0.001 * (co2ppm - 410))'},
 {'supplier': {'name': 'Dinitrogen monoxide',
   'operator': 'equals',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process'},
  'value': '265 * (1 + 0.0005 * (co2ppm - 410))'}]

We can instantiate the EdgeLCIA() class as usual, except that we need to pass the parameters as a dictionary.
Then we proceed to the mapping steps. Finally, we iterate over the scenarios and evaluate the CFs.

In [11]:

import bw2data
from edges import EdgeLCIA

# Select an activity from the LCA database
bw2data.projects.set_current("ecoinvent-3.10-cutoff")
act = bw2data.Database("ecoinvent-3.10.1-cutoff").random()
print(act)

# Define the new allowed function
allowed_funcs = {"GWP": GWP}

# Define scenario parameters (e.g., atmospheric CO₂ concentration and time horizon)
params = {
    "H": 100,
    "C_CH4": [1991, 2000, 2100], # possible ppb concentrations in the future
    "C_N2O": [332, 340, 350] # possible ppb concentrations in the future
}

# Define an LCIA method (symbolic CF expressions stored in JSON)
method = ('GWP', 'scenario-dependent', '100 years')

# Initialize LCIA
lcia = EdgeLCIA(
   demand={act: 1},
   filepath="lcia_example_4.json",
   parameters=params,
   allowed_functions=allowed_funcs
)

# Perform inventory calculations (once)
lcia.lci()

# Map exchanges to CF entries (once)
lcia.map_exchanges()

# Optionally, resolve geographic overlaps and disaggregations (once)
lcia.map_aggregate_locations()
lcia.map_dynamic_locations()
lcia.map_remaining_locations_to_global()

# Run scenarios efficiently
results = []
for idx in range(lcia.scenario_length):
    lcia.evaluate_cfs(idx)
    lcia.lcia()
    df = lcia.generate_cf_table()

    scenario_result = {
        "scenario": idx,
        "score": lcia.score,
        "CF_table": df
    }
    results.append(scenario_result)

    print(f"Scenario {idx+1}: Impact = {lcia.score}")

'market for selective coat, copper sheet, black majic' (square meter, GLO, None)


/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 4.83e+13)
  warnings.warn(msg, UmfpackWarning)


Identifying eligible exchanges...


100%|██████████| 3/3 [00:00<00:00, 35.90it/s]

Handling static regions...


Handling dynamic regions...
Handling remaining exchanges...


Processing remaining global edges (pass 2): 100%|██████████| 323142/323142 [00:00<00:00, 660157.96it/s]


Scenario 1: Impact = 1.789613057709253
Scenario 2: Impact = 1.7890401821120638
Scenario 3: Impact = 1.7852831824280173


In [12]:
scenario_result["CF_table"]["CF"].unique()

array([  1.        ,  29.24847381, 292.626264  ])